In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
print("CUDA_VISIBLE_DEVICES =", os.environ["CUDA_VISIBLE_DEVICES"])

!pip install python-docx seaborn pillow --quiet

from google.colab import drive
drive.mount('/content/drive')

import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from docx.shared import Inches
import copy

device = torch.device('cpu')
print("Running on device:", device)

random.seed(42)
np.random.seed(42)

DATA_ROOT  = '/content/drive/MyDrive/dp/data'
OUTPUT_DIR = '/content/drive/MyDrive/dp/output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

BATCH_SIZE    = 32
NUM_EPOCHS    = 30
LEARNING_RATE = 1e-4

BEST_MODEL_PTH = os.path.join(OUTPUT_DIR, 'best_model.pth')
BEST_ACC_TXT   = os.path.join(OUTPUT_DIR, 'best_val_acc.txt')
CLF_TXT        = os.path.join(OUTPUT_DIR, 'classification_report.txt')
CM_PNG         = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')

class Transform:
    def __init__(self):
        self.mean = np.array([0.485]*3, dtype=np.float32)
        self.std  = np.array([0.229]*3, dtype=np.float32)
    def __call__(self, img: Image.Image):
        img = img.resize((224,224))
        arr = np.array(img.convert('RGB'), dtype=np.float32)/255.0
        arr = (arr - self.mean)/self.std
        return torch.from_numpy(arr).permute(2,0,1)

transform = Transform()

class AnnotDataset(Dataset):
    def __init__(self, root, transform):
        self.samples = []
        self.classes = sorted(d for d in os.listdir(root) if os.path.isdir(os.path.join(root,d)))
        self.cl2i    = {c:i for i,c in enumerate(self.classes)}
        for c in self.classes:
            for fn in os.listdir(os.path.join(root,c)):
                if fn.lower().endswith(('.jpg','.jpeg','.png')):
                    self.samples.append((os.path.join(root,c,fn), self.cl2i[c]))
        self.transform = transform
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        p,l = self.samples[idx]
        img = Image.open(p)
        return self.transform(img), l

full_ds = AnnotDataset(DATA_ROOT, transform)
NUM_CLASSES = len(full_ds.classes)
n = len(full_ds)
n_train = int(0.8*n)
n_val   = n - n_train
train_ds, val_ds = random_split(full_ds, [n_train, n_val])
train_ld = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_ld   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128,256,3,padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(256,512,3,padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.AdaptiveAvgPool2d((7,7))
        )
    def forward(self,x):
        return self.f(x)

class HybridCNNLSTM(nn.Module):
    def __init__(self, num_classes, hid=256, layers=2, dp=0.5):
        super().__init__()
        self.cnn = SimpleCNN()
        self.lstm = nn.LSTM(512, hid, num_layers=layers,
                            batch_first=True, bidirectional=True, dropout=dp)
        self.head = nn.Sequential(
            nn.Linear(hid*2,128), nn.ReLU(), nn.Dropout(dp),
            nn.Linear(128,num_classes)
        )
    def forward(self,x):
        f = self.cnn(x)
        B,C,H,W = f.shape
        seq = f.view(B,C,H*W).permute(0,2,1)
        _,(h,_) = self.lstm(seq)
        h = torch.cat([h[-2],h[-1]], dim=1)
        return self.head(h)

model = HybridCNNLSTM(NUM_CLASSES).cpu()
crit  = nn.CrossEntropyLoss()
opt   = optim.Adam(model.parameters(), lr=LEARNING_RATE)

best_val=0.0
best_wts=copy.deepcopy(model.state_dict())
hist={'tl':[],'ta':[],'vl':[],'va':[]}

for e in range(1, NUM_EPOCHS+1):
    model.train()
    t_loss=t_corr=t_cnt=0
    for x,y in train_ld:
        logits = model(x)
        loss   = crit(logits, y)
        opt.zero_grad(); loss.backward(); opt.step()
        t_loss+= loss.item()*x.size(0)
        preds  = logits.argmax(1)
        t_corr += (preds==y).sum().item()
        t_cnt  += y.size(0)
    tl,ta = t_loss/t_cnt, t_corr/t_cnt

    model.eval()
    v_loss=v_corr=v_cnt=0
    with torch.no_grad():
        for x,y in val_ld:
            lo = model(x)
            l  = crit(lo,y)
            v_loss+= l.item()*x.size(0)
            pr = lo.argmax(1)
            v_corr += (pr==y).sum().item()
            v_cnt  += y.size(0)
    vl,va = v_loss/v_cnt, v_corr/v_cnt

    hist['tl'].append(tl); hist['ta'].append(ta)
    hist['vl'].append(vl); hist['va'].append(va)

    if va>best_val:
        best_val=va
        best_wts=copy.deepcopy(model.state_dict())
        torch.save(best_wts, BEST_MODEL_PTH)

    print(f"Epoch {e}/{NUM_EPOCHS} → Train: {tl:.4f}/{ta:.4f}  Val: {vl:.4f}/{va:.4f}")

with open(BEST_ACC_TXT,'w') as f: f.write(f"{best_val:.4f}\n")

plt.figure(); plt.plot(hist['tl'], label='Train Loss'); plt.plot(hist['vl'], label='Val Loss')
plt.legend(); plt.savefig(os.path.join(OUTPUT_DIR,'loss_curve.png')); plt.close()
plt.figure(); plt.plot(hist['ta'], label='Train Acc'); plt.plot(hist['va'], label='Val Acc')
plt.legend(); plt.savefig(os.path.join(OUTPUT_DIR,'acc_curve.png')); plt.close()

model.load_state_dict(best_wts)
all_y, all_p = [],[]
with torch.no_grad():
    for x,y in val_ld:
        preds = model(x).argmax(1).numpy()
        all_p.extend(preds); all_y.extend(y.numpy())

cm = confusion_matrix(all_y, all_p)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=full_ds.classes, yticklabels=full_ds.classes)
plt.savefig(CM_PNG); plt.close()

rep = classification_report(all_y, all_p, target_names=full_ds.classes)
with open(CLF_TXT,'w') as f: f.write(rep)

doc=Document()
doc.add_heading('Pneumonia Detection Report', level=1)
doc.add_paragraph(f"Best Val Acc: {best_val:.4f}")
doc.add_heading('Confusion Matrix', level=2); doc.add_picture(CM_PNG, width=Inches(5))
doc.add_heading('Classification Report', level=2); doc.add_paragraph(rep)
doc.add_heading('Training/Validation Curves', level=2)
doc.add_picture(os.path.join(OUTPUT_DIR,'loss_curve.png'), width=Inches(5))
doc.add_picture(os.path.join(OUTPUT_DIR,'acc_curve.png'), width=Inches(5))
doc.save(os.path.join(OUTPUT_DIR,'report.docx'))

print("Done! All outputs in", OUTPUT_DIR)
